In [3]:
# 01_create_datasets.py
from clearml import Task, Dataset
import pandas as pd
import numpy as np
import os
import shutil

# ======= 1. Создаем Task для управления датасетами =======
task = Task.init(
    project_name="Customer_Return_Prediction",
    task_name="Dataset Creation and Versioning",
    task_type=Task.TaskTypes.data_processing
)

# Логируем параметры
task.set_parameter("dataset_versioning", {
    "raw_data_source": "transaction_.csv",
    "train_test_split": "temporal_split",
    "train_period": "until 2019-09-01",
    "test_period": "2019-09-01 to 2019-10-01"
})

# ======= 2. ВЕРСИЯ 1: Сырые данные (минимально очищенные) =======
print("🔹 СОЗДАНИЕ ДАТАСЕТА V1: RAW DATA")

# Загружаем и минимально очищаем данные
transaction = pd.read_csv('transaction_.csv')
rename_dict = {'clientID': 'client', 'trDte': 'visit_date', 'itemGroup': 'item_group'}
transaction = transaction.rename(columns=rename_dict)
transaction['visit_date'] = pd.to_datetime(transaction['visit_date'], format='%d.%m.%Y')

# Сохраняем очищенные данные
os.makedirs('datasets', exist_ok=True)
transaction.to_csv('datasets/raw_cleaned.csv', index=False)

# Создаем датасет V1
dataset_v1 = Dataset.create(
    dataset_name="customer_data_raw",
    dataset_project="Customer_Return_Prediction",
    dataset_tags=["v1.0", "raw", "cleaned"]
)

# Исправлено: передаем строку, а не список
dataset_v1.add_files('datasets/raw_cleaned.csv')
dataset_v1.set_metadata({
    "description": "Минимально очищенные транзакционные данные",
    "processing_steps": ["loading", "column_renaming", "date_parsing"],
    "rows": len(transaction),
    "columns": list(transaction.columns),
    "date_range": f"{transaction['visit_date'].min().date()} to {transaction['visit_date'].max().date()}"
})

dataset_v1.upload()
dataset_v1.finalize()

print(f"✅ Dataset V1 создан: {dataset_v1.id}")
task.upload_artifact("raw_data_sample", transaction.head(100))

# ======= 3. ВЕРСИЯ 2: Обучающие и тестовые выборки =======
print("\n🔹 СОЗДАНИЕ ДАТАСЕТА V2: TRAIN/TEST SPLITS")

# Используем вашу логику для создания train/test
def calculate_client_profile_at_date(df, observation_end_date):
    """Ваша функция из предыдущего кода"""
    obs_date = pd.to_datetime(observation_end_date)
    visits = df[df['visit_date'] < obs_date].copy()
    
    if visits.empty:
        return pd.DataFrame()
    
    visits['is_weekend'] = visits['visit_date'].dt.dayofweek >= 5
    
    agg = visits.groupby('client').agg(
        last_visit=('visit_date', 'max'),
        visits=('visit_date', 'count'),
        unique_visits=('visit_date', 'nunique'),
        amount=('amount', 'sum'),
        quantity=('quantity', 'sum'),
        items=('item', 'nunique'),
        weekends=('is_weekend', 'sum')
    ).reset_index()
    
    agg['Recency'] = (obs_date - agg['last_visit']).dt.days
    agg['Frequency'] = agg['unique_visits']
    agg['Monetary'] = agg['amount']
    agg['avg_check'] = agg['amount'] / agg['Frequency']
    agg['avg_items'] = agg['quantity'] / agg['Frequency']
    
    last_amount = visits.loc[visits.groupby('client')['visit_date'].idxmax()]
    agg = agg.merge(last_amount[['client', 'amount']].rename(
        columns={'amount': 'last_visit_amount'}), on='client')
    
    return agg

def mark_events(df, result_start_date, result_end_date):
    """Ваша функция разметки событий"""
    start_date = pd.to_datetime(result_start_date)
    end_date = pd.to_datetime(result_end_date)
    
    base_clients = df['client'].unique()
    period_mask = (df['visit_date'] >= start_date) & (df['visit_date'] < end_date)
    active_clients = df.loc[period_mask, 'client'].unique()
    
    return pd.DataFrame({
        'client': base_clients,
        'event': [1 if client in active_clients else 0 for client in base_clients]
    })

# Создаем train данные
train_profile = calculate_client_profile_at_date(transaction, '2019-09-01')
train_events = mark_events(transaction, '2019-09-01', '2019-10-01')
train_df = train_profile.merge(train_events, on='client', how='inner')
train_df['event'] = train_df['event'].astype(int)
train_df = train_df.drop(columns=[col for col in train_df.columns if 'date' in col.lower()], errors='ignore')

# Создаем test данные
test_profile = calculate_client_profile_at_date(transaction, '2019-10-01')
test_events = mark_events(transaction, '2019-10-01', '2019-11-01')
test_df = test_profile.merge(test_events, on='client', how='inner')
test_df['event'] = test_df['event'].astype(int)
test_df = test_df.drop(columns=[col for col in test_df.columns if 'date' in col.lower()], errors='ignore')

# Сохраняем файлы
train_df.to_csv('datasets/train_data_v2.csv', index=False)
test_df.to_csv('datasets/test_data_v2.csv', index=False)

# Создаем датасет V2 с наследованием от V1
dataset_v2 = Dataset.create(
    dataset_name="customer_data_train_test",
    dataset_project="Customer_Return_Prediction",
    dataset_tags=["v2.0", "processed", "train_test_split"],
    parent_datasets=[dataset_v1.id]  # Наследуем от V1!
)

# Исправлено: добавляем файлы по одному
dataset_v2.add_files('datasets/train_data_v2.csv')
dataset_v2.add_files('datasets/test_data_v2.csv')

# Или можно добавить всю папку:
# dataset_v2.add_files('datasets/')

dataset_v2.set_metadata({
    "description": "Обработанные данные с train/test разбиением",
    "parent_dataset": dataset_v1.id,
    "train_samples": len(train_df),
    "test_samples": len(test_df),
    "features_count": len(train_df.columns) - 1,  # без target
    "target_distribution_train": {
        "returned": int(train_df['event'].sum()),
        "not_returned": int(len(train_df) - train_df['event'].sum()),
        "return_rate": float(train_df['event'].mean())
    },
    "target_distribution_test": {
        "returned": int(test_df['event'].sum()),
        "not_returned": int(len(test_df) - test_df['event'].sum()),
        "return_rate": float(test_df['event'].mean())
    }
})

dataset_v2.upload()
dataset_v2.finalize()

print(f"✅ Dataset V2 создан: {dataset_v2.id}")
print(f"   Наследует от: {dataset_v1.id}")

# ======= 4. Логируем статистики в Task =======
task.upload_artifact("train_data_sample", train_df.head(100))
task.upload_artifact("test_data_sample", test_df.head(100))

stats = {
    "datasets_created": {
        "v1_raw": dataset_v1.id,
        "v2_train_test": dataset_v2.id
    },
    "data_statistics": {
        "raw_rows": len(transaction),
        "train_rows": len(train_df),
        "test_rows": len(test_df),
        "train_features": list(train_df.columns),
        "train_target_balance": f"{train_df['event'].mean():.1%}",
        "test_target_balance": f"{test_df['event'].mean():.1%}"
    }
}

task.upload_artifact("dataset_statistics", stats)

print("\n" + "="*60)
print("✅ ВСЕ ДАТАСЕТЫ СОЗДАНЫ И ВЕРСИОНИРОВАНЫ!")
print("="*60)
print(f"V1 (Raw):      {dataset_v1.id}")
print(f"V2 (Train/Test): {dataset_v2.id}")
print(f"Lineage: V1 → V2")

task.close()

🔹 СОЗДАНИЕ ДАТАСЕТА V1: RAW DATA
ClearML results page: https://app.clear.ml/projects/c6b27df2c1d343f5b8ded148b93dfe5e/experiments/039ed9d78d9e49b5a7efbd604c8ce0b5/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/c6b27df2c1d343f5b8ded148b93dfe5e/experiments/039ed9d78d9e49b5a7efbd604c8ce0b5
Displaying metadata in the UI is only supported for pandas Dataframes for now. Skipping!
Uploading dataset changes (1 files compressed to 9.9 MiB) to https://files.clear.ml


██████████████████████████████████ 100% | 9.90/9.9 MB [00:02<00:00,  3.84MB/s]: 


File compression and upload completed: total size 9.9 MiB, 1 chunk(s) stored (average size 9.9 MiB)
✅ Dataset V1 создан: 039ed9d78d9e49b5a7efbd604c8ce0b5

🔹 СОЗДАНИЕ ДАТАСЕТА V2: TRAIN/TEST SPLITS
ClearML results page: https://app.clear.ml/projects/6c22464c761a410b84f5e312a13b6663/experiments/3520fae4f5044740872128c239c09ff9/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/6c22464c761a410b84f5e312a13b6663/experiments/3520fae4f5044740872128c239c09ff9
Displaying metadata in the UI is only supported for pandas Dataframes for now. Skipping!
Uploading dataset changes (2 files compressed to 3.86 MiB) to https://files.clear.ml
File compression and upload completed: total size 3.86 MiB, 1 chunk(s) stored (average size 3.86 MiB)
✅ Dataset V2 создан: 3520fae4f5044740872128c239c09ff9
   Наследует от: 039ed9d78d9e49b5a7efbd604c8ce0b5

✅ ВСЕ ДАТАСЕТЫ СОЗДАНЫ И ВЕРСИОНИРОВАНЫ!
V1 (Raw):      039ed9d78d9e49b5a7efbd604c8ce0b5
V2 (Train/Test): 3520fae4f5044740872128c239c09ff9
Linea

In [4]:
# 02_logistic_regression_experiment.py
from clearml import Task, Dataset
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pickle
import joblib
import os

# ======= 1. Создаем Task для эксперимента =======
task = Task.init(
    project_name="Customer_Return_Prediction",
    task_name="Logistic Regression Experiment",
    task_type=Task.TaskTypes.training
)

# ======= 2. Загружаем датасет V2 через ClearML =======
print("📥 Загрузка данных из ClearML Dataset...")
dataset_v2 = Dataset.get(
    dataset_name="customer_data_train_test",
    dataset_project="Customer_Return_Prediction",
    dataset_tags=["v2.0", "processed", "train_test_split"],
    only_completed=True
)

# Скачиваем данные локально
local_path = dataset_v2.get_local_copy()
print(f"Данные скачаны в: {local_path}")

# Загружаем train/test файлы
train_path = os.path.join(local_path, 'train_data_v2.csv')
test_path = os.path.join(local_path, 'test_data_v2.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(f"Train data: {train_df.shape}")
print(f"Test data: {test_df.shape}")

# ======= 3. Подготовка данных =======
X_train = train_df.drop(columns=['client', 'event'], errors='ignore')
y_train = train_df['event']
X_test = test_df.drop(columns=['client', 'event'], errors='ignore')
y_test = test_df['event']

# Оставляем только числовые признаки
X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

# Синхронизация колонок
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# ======= 4. Гиперпараметры модели =======
hyperparameters = {
    "model_type": "LogisticRegression",
    "penalty": "l2",           # L2 регуляризация
    "C": 1.0,                  # Обратная сила регуляризации
    "solver": "lbfgs",         # Алгоритм оптимизации
    "max_iter": 1000,          # Максимальное количество итераций
    "random_state": 42,
    "class_weight": None,      # Веса классов
    "fit_intercept": True      # Добавлять intercept
}

# Логируем гиперпараметры в ClearML (3.4)
task.connect(hyperparameters)
print("✅ Гиперпараметры залогированы в ClearML")

# ======= 5. Обучение модели =======
print("\n🔧 Обучение Logistic Regression...")
model = LogisticRegression(
    penalty=hyperparameters["penalty"],
    C=hyperparameters["C"],
    solver=hyperparameters["solver"],
    max_iter=hyperparameters["max_iter"],
    random_state=hyperparameters["random_state"],
    class_weight=hyperparameters["class_weight"],
    fit_intercept=hyperparameters["fit_intercept"]
)

model.fit(X_train, y_train)

# ======= 6. Прогнозирование и оценка =======
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

# Вычисляем метрики
roc_auc = roc_auc_score(y_test, y_pred_proba)
accuracy = (y_pred == y_test).mean()
precision = ((y_pred == 1) & (y_test == 1)).sum() / max((y_pred == 1).sum(), 1)
recall = ((y_pred == 1) & (y_test == 1)).sum() / max(y_test.sum(), 1)
f1 = 2 * precision * recall / max(precision + recall, 1e-10)

# Логируем метрики в ClearML (3.5)
task.get_logger().report_scalar(
    title="Model Metrics",
    series="ROC-AUC",
    value=roc_auc,
    iteration=0
)

task.get_logger().report_scalar(
    title="Model Metrics",
    series="Accuracy",
    value=accuracy,
    iteration=0
)

task.get_logger().report_scalar(
    title="Model Metrics", 
    series="F1-Score",
    value=f1,
    iteration=0
)

print(f"\n📊 МЕТРИКИ МОДЕЛИ:")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# ======= 7. Визуализация =======
# ROC-кривая
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {roc_auc:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Model', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
task.get_logger().report_matplotlib_figure(
    title="ROC Curve",
    series="Logistic Regression",
    figure=plt.gcf(),
    iteration=0
)
plt.close()

# Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
task.get_logger().report_matplotlib_figure(
    title="Confusion Matrix",
    series="Logistic Regression", 
    figure=plt.gcf(),
    iteration=0
)
plt.close()

# ======= 8. Сохранение модели как артефакта (3.3) =======
print("\n💾 Сохранение модели...")
os.makedirs('models', exist_ok=True)
model_filename = 'models/logistic_regression_model.pkl'

# Сохраняем через joblib (лучше для sklearn моделей)
joblib.dump(model, model_filename)
print(f"Модель сохранена: {model_filename}")

# Загружаем как артефакт в ClearML
task.upload_artifact(
    name="logistic_regression_model",
    artifact_object=model_filename,
    metadata={
        "model_type": "LogisticRegression",
        "hyperparameters": hyperparameters,
        "metrics": {
            "roc_auc": roc_auc,
            "accuracy": accuracy,
            "f1_score": f1
        },
        "dataset_version": dataset_v2.id
    }
)

# Также сохраняем через pickle для полноты
with open('models/logistic_regression_model_pickle.pkl', 'wb') as f:
    pickle.dump(model, f)

# ======= 9. Важность признаков =======
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': abs(model.coef_[0])
}).sort_values('coefficient', ascending=False)

task.upload_artifact("feature_importance", feature_importance)

# ======= 10. Итоговая информация =======
print("\n" + "="*60)
print("✅ ЭКСПЕРИМЕНТ ЗАВЕРШЕН!")
print("="*60)
print(f"Task ID: {task.id}")
print(f"Dataset used: {dataset_v2.id}")
print(f"Model saved: {model_filename}")
print(f"ROC-AUC: {roc_auc:.4f}")

task.close()

Can't get url information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages


ClearML Task: created new task id=58f1a07bd9f74a22881b7e2417b1469b


Can't get branch information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages
Can't get commit information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages
Can't get diff information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages


ClearML results page: https://app.clear.ml/projects/ebbc67863e744623a4f3e7490e757670/experiments/58f1a07bd9f74a22881b7e2417b1469b/output/log
📥 Загрузка данных из ClearML Dataset...
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


██████████████████████████████████ 100% | 9.90/9.9 MB [00:01<00:00,  9.51MB/s]: C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages\tqdm\std.py:636: TqdmWarning:

clamping frac to range [0, 1]

██████████████████████████████████ 100% | 9.90/9.9 MB [00:01<00:00,  9.46MB/s]: 


Данные скачаны в: C:/Users/Ksenia/.clearml/cache/storage_manager/datasets/ds_3520fae4f5044740872128c239c09ff9
Train data: (39906, 15)
Test data: (41196, 15)
✅ Гиперпараметры залогированы в ClearML

🔧 Обучение Logistic Regression...


C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning:

lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression




📊 МЕТРИКИ МОДЕЛИ:
ROC-AUC: 0.7980
Accuracy: 0.8274
Precision: 0.6385
Recall: 0.1966
F1-Score: 0.3006

💾 Сохранение модели...
Модель сохранена: models/logistic_regression_model.pkl

✅ ЭКСПЕРИМЕНТ ЗАВЕРШЕН!
Task ID: 58f1a07bd9f74a22881b7e2417b1469b
Dataset used: 3520fae4f5044740872128c239c09ff9
Model saved: models/logistic_regression_model.pkl
ROC-AUC: 0.7980


In [ ]:
# 03_xgboost_experiment.py
from clearml import Task, Dataset
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import joblib
import os

# ======= 1. Создаем Task для эксперимента =======
task = Task.init(
    project_name="Customer_Return_Prediction", 
    task_name="XGBoost Experiment",
    task_type=Task.TaskTypes.training
)

# ======= 2. Загружаем датасет V2 =======
print("📥 Загрузка данных из ClearML Dataset...")
dataset_v2 = Dataset.get(
    dataset_name="customer_data_train_test",
    dataset_project="Customer_Return_Prediction",
    dataset_tags=["v2.0", "processed", "train_test_split"],
    only_completed=True
)

local_path = dataset_v2.get_local_copy()
train_path = os.path.join(local_path, 'train_data_v2.csv')
test_path = os.path.join(local_path, 'test_data_v2.csv')

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# ======= 3. Подготовка данных =======
X_train = train_df.drop(columns=['client', 'event'], errors='ignore')
y_train = train_df['event']
X_test = test_df.drop(columns=['client', 'event'], errors='ignore')
y_test = test_df['event']

X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# ======= 4. Гиперпараметры XGBoost =======
hyperparameters = {
    "model_type": "XGBClassifier",
    "n_estimators": 150,           # Увеличили с 100 (в прошлом коде)
    "max_depth": 5,                # Увеличили с 3
    "learning_rate": 0.05,         # Уменьшили с 0.1
    "subsample": 0.8,              # Добавили subsample
    "colsample_bytree": 0.8,       # Добавили colsample
    "reg_alpha": 0.1,              # L1 регуляризация
    "reg_lambda": 1.0,             # L2 регуляризация
    "random_state": 42,
    "eval_metric": "logloss",
    "use_label_encoder": False
}

# Логируем гиперпараметры (3.4)
task.connect(hyperparameters)
print("✅ Гиперпараметры XGBoost залогированы")

# ======= 5. Обучение модели =======
print("\n🔧 Обучение XGBoost...")
model = XGBClassifier(
    n_estimators=hyperparameters["n_estimators"],
    max_depth=hyperparameters["max_depth"],
    learning_rate=hyperparameters["learning_rate"],
    subsample=hyperparameters["subsample"],
    colsample_bytree=hyperparameters["colsample_bytree"],
    reg_alpha=hyperparameters["reg_alpha"],
    reg_lambda=hyperparameters["reg_lambda"],
    random_state=hyperparameters["random_state"],
    eval_metric=hyperparameters["eval_metric"],
    use_label_encoder=hyperparameters["use_label_encoder"]
)

model.fit(X_train, y_train)

# ======= 6. Оценка модели =======
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba)
accuracy = (y_pred == y_test).mean()

# Логируем метрики (3.5)
task.get_logger().report_scalar(
    title="Model Metrics",
    series="ROC-AUC",
    value=roc_auc,
    iteration=0
)

task.get_logger().report_scalar(
    title="Model Metrics",
    series="Accuracy", 
    value=accuracy,
    iteration=0
)

print(f"\n📊 МЕТРИКИ XGBoost:")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# ======= 7. Сравнение с Logistic Regression =======
# Можно загрузить метрики из предыдущего эксперимента
print("\n🔍 СРАВНЕНИЕ С LOGISTIC REGRESSION:")
print("(В реальном сценарии нужно загрузить метрики из предыдущего Task)")
print("XGBoost обычно показывает лучшие результаты на сложных данных")

# ======= 8. Визуализация =======
# ROC-кривая
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'XGBoost (AUC = {roc_auc:.3f})', linewidth=2, color='green')
plt.plot([0, 1], [0, 1], 'k--', label='Random Model', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - XGBoost')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
task.get_logger().report_matplotlib_figure(
    title="ROC Curve",
    series="XGBoost",
    figure=plt.gcf(),
    iteration=0
)
plt.close()

# Важность признаков
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
plt.barh(importance_df['feature'][:15], importance_df['importance'][:15])
plt.xlabel('Feature Importance')
plt.title('Top 15 Feature Importance - XGBoost')
plt.gca().invert_yaxis()
task.get_logger().report_matplotlib_figure(
    title="Feature Importance",
    series="XGBoost",
    figure=plt.gcf(),
    iteration=0
)
plt.close()

# ======= 9. Сохранение модели (3.3) =======
print("\n💾 Сохранение XGBoost модели...")
model_filename = 'models/xgboost_model.pkl'
joblib.dump(model, model_filename)

task.upload_artifact(
    name="xgboost_model",
    artifact_object=model_filename,
    metadata={
        "model_type": "XGBClassifier",
        "hyperparameters": hyperparameters,
        "metrics": {
            "roc_auc": roc_auc,
            "accuracy": accuracy
        },
        "dataset_version": dataset_v2.id,
        "top_features": importance_df['feature'].head(5).tolist()
    }
)

# ======= 10. Выводы для пояснительной записки =======
print("\n" + "="*60)
print("📝 ДЛЯ ПОЯСНИТЕЛЬНОЙ ЗАПИСКИ:")
print("="*60)
print("\nГиперпараметры, которые варьировались:")
print("1. n_estimators: 100 → 150 (увеличение количества деревьев)")
print("2. max_depth: 3 → 5 (увеличение глубины деревьев)")
print("3. learning_rate: 0.1 → 0.05 (уменьшение скорости обучения)")
print("4. Добавлены регуляризации: reg_alpha, reg_lambda")
print("5. Добавлены: subsample, colsample_bytree")

print("\nКак зафиксировано в ClearML:")
print("- Использован Task.connect() для логирования гиперпараметров")
print("- Параметры видны во вкладке Configuration")
print("- Каждый эксперимент имеет уникальный набор параметров")

print("\nСравнительный анализ (нужно выполнить оба эксперимента):")
print("1. Logistic Regression: проще, интерпретируем, быстрее")
print("2. XGBoost: сложнее, обычно лучше точность, медленнее обучение")
print("3. Сравнить ROC-AUC из обоих экспериментов")
print("4. Проанализировать важность признаков в обеих моделях")

print("\n" + "="*60)
print("✅ ЭКСПЕРИМЕНТ XGBoost ЗАВЕРШЕН!")
print("="*60)

task.close()

Can't get url information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages


ClearML Task: created new task id=eb2fabf954da4fd1bac6f2017e55231f


Can't get branch information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages
Can't get commit information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages
Can't get diff information for git repo in C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages


ClearML results page: https://app.clear.ml/projects/ebbc67863e744623a4f3e7490e757670/experiments/eb2fabf954da4fd1bac6f2017e55231f/output/log
📥 Загрузка данных из ClearML Dataset...
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
✅ Гиперпараметры XGBoost залогированы

🔧 Обучение XGBoost...


C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:199: UserWarning:

[18:31:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.





📊 МЕТРИКИ XGBoost:
ROC-AUC: 0.8075
Accuracy: 0.8337

🔍 СРАВНЕНИЕ С LOGISTIC REGRESSION:
(В реальном сценарии нужно загрузить метрики из предыдущего Task)
XGBoost обычно показывает лучшие результаты на сложных данных


C:\Users\Ksenia\AppData\Roaming\Python\Python311\site-packages\clearml\utilities\plotlympl\renderer.py:209: UserWarning:

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.




💾 Сохранение XGBoost модели...

📝 ДЛЯ ПОЯСНИТЕЛЬНОЙ ЗАПИСКИ:

Гиперпараметры, которые варьировались:
1. n_estimators: 100 → 150 (увеличение количества деревьев)
2. max_depth: 3 → 5 (увеличение глубины деревьев)
3. learning_rate: 0.1 → 0.05 (уменьшение скорости обучения)
4. Добавлены регуляризации: reg_alpha, reg_lambda
5. Добавлены: subsample, colsample_bytree

Как зафиксировано в ClearML:
- Использован Task.connect() для логирования гиперпараметров
- Параметры видны во вкладке Configuration
- Каждый эксперимент имеет уникальный набор параметров

Сравнительный анализ (нужно выполнить оба эксперимента):
1. Logistic Regression: проще, интерпретируем, быстрее
2. XGBoost: сложнее, обычно лучше точность, медленнее обучение
3. Сравнить ROC-AUC из обоих экспериментов
4. Проанализировать важность признаков в обеих моделях

✅ ЭКСПЕРИМЕНТ XGBoost ЗАВЕРШЕН!


Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000014A86C80DD0>: Failed to establish a new connection: [WinError 10065] Сделана попытка выполнить операцию на сокете для недоступного хоста')': /v2.23/tasks.ping
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000014AE1499050>: Failed to resolve 'api.clear.ml' ([Errno 11001] getaddrinfo failed)")': /v2.23/tasks.ping
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000014A82B7A810>: Failed to resolve 'api.clear.ml' ([Errno 11001] getaddrinfo failed)")': /v2.23/tasks.ping
Retrying (Retry(total=235, connect=235, read=240, redirect=240, status=240)) after connection bro